## 📌 사전 기반의 후처리

- 사전을 구축하여 전처리한 테스트 데이터([test_preprocess.csv](../data/test_preprocess.csv))를 바탕으로 전체 리뷰를 복원한 데이터([submission_total.csv](../submissions/submission_total.csv))를 후처리합니다.
- 본 대회의 경우에는 복원된 리뷰의 길이가 일치해야 하는 부분이 중요하기 때문에 길이를 맞춰주는 작업이 중요합니다.
- 따라서 길이가 일치하지 않는 부분에 대해서는 테스트 데이터에 있는 부분을 그대로 활용합니다.

❗️NEXT STEP : [복원된 리뷰를 바탕을 난독화된 리뷰 문장 단위로 분할](./split_test_sentences.ipynb)

### 🏷️ 대응되는 단어 기반 후처리

In [1]:
import pandas as pd

test_df = pd.read_csv("../data/test_preprocess.csv", encoding="utf-8-sig") # 난독화된 리뷰를 가져오기 위한 테스트 파일
submit_df = pd.read_csv("../submissions/submission_total.csv", encoding="utf-8-sig") # 후처리할 복원된 리뷰를 가져오기 위한 제출 파일

In [2]:
import re

# 단어와 공백을 분리하는 함수
def split_with_spaces(text):
    # 공백(1개 이상)과 단어를 분리
    return re.split(r'(\s+)', text)

# 데이터프레임에 적용
test_df["text"] = test_df["input"].apply(split_with_spaces)
submit_df["text"] = submit_df["output"].apply(split_with_spaces)

In [3]:
test_df["text"].head()

0    [너무너무,  , 만족숭러윤,  , 호데리에요.,  , 푸싸눼,  , 옰면,  , ...
1    [프론트갸,  , 엎코,,  , 좀식또,  , 업읍머,,  , 일반,  , 잎츔민든...
2    [진짜,  , 붉친절해요.,  , 삶면서,  , 머물려던,  , 호텔,  , 중웨,...
3    [붊,  , 맛집~~,  , 그런뎨,  , 방음잃,  , 뮈흙퍄네요.,  , 중간,...
4    [빻,  , 상태는,  , 진짜,  , 폐헐,  , 칙젓인데,  , 전망은,  , ...
Name: text, dtype: object

In [4]:
submit_df["text"].head()

0    [너무너무,  , 만족스러운,  , 호텔이에요.,  , 부산에,  , 오면,  , ...
1    [프론트가,  , 없고,,  , 조식도,  , 없으며,,  , 일반,  , 입주민들...
2    [진짜,  , 불친절해요.,  , 살면서,  , 머물렀던,  , 호텔,  , 중에,...
3    [뷰,  , 맛집~~,  , 그런데,  , 방음이,  , 미흡하네요.,  , 층간,...
4    [방,  , 상태는,  , 진짜,  , 폐허,  , 직전인데,  , 전망은,  , ...
Name: text, dtype: object

### 🏷️ 대응되는 단어의 길이가 일치하지 않은 경우

In [5]:
for idx, (input_text, output_text) in enumerate(zip(test_df["text"], submit_df["text"])):
    for i, (input_word, output_word) in enumerate(zip(input_text, output_text)):
        # 글자 수가 다른 단어들을 추출
        if len(input_word) != len(output_word):
            output_text[i] = input_word

### 🏷️ 전체 문장 길이가 일치하지 않을 경우

In [6]:
outputs = []
for i, (input_text, output_text) in enumerate(zip(test_df["input"], submit_df["text"])):
    # output_text 리스트를 문자열로 합침
    combined_output = "".join(output_text)
    if len(combined_output) < len(input_text):
        # 처음부터 복원된 리뷰의 길이가 난독화된 리뷰의 길이보다 짧다면 남은 길이만큼 난독화된 리뷰를 그대로 이어 붙이기
        combined_output = combined_output + input_text[len(combined_output):]
    elif len(combined_output) > len(input_text):
        # 처음부터 복원된 리뷰의 길이가 난독화된 리뷰의 길이보다 길다면 난독화된 리뷰의 길이만큼만 잘라내기
        combined_output = combined_output[:len(input_text)]
        
    outputs.append(combined_output)

### 🏷️ 후처리된 파일 저장

In [7]:
# 제출 파일에 대입
submit_df["output"] = outputs
submit_df[["ID", "output"]].to_csv("../submissions/submission_postprocess.csv", index=False, encoding="utf-8-sig")